In [1]:
import matplotlib.colors
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

import seaborn as sns
sns.set_style("whitegrid")

import urllib3
import os
import matplotlib
import pickle

from sklearn.preprocessing import LabelEncoder

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

Urllib3 version must be 1.24 to make this work

In [2]:
urllib3.__version__

'1.24'

In [35]:
!pip install --upgrade --force urllib3==1.24

  Using cached urllib3-1.24-py2.py3-none-any.whl (117 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
twine 3.8.0 requires urllib3>=1.26.0, but you have urllib3 1.24 which is incompatible.


In [3]:
http = urllib3.PoolManager()

In [4]:
boroughs = {
    'Camden': 1946157246,
    'City of London': 1946157247,
    'Hackney': 1946157248,
    'Haringey': 1946157250,
    'Islington': 1946157251,
    'Kensington and Chelsea': 1946157252,
    'Lambeth': 1946157253,
    'Lewisham': 1946157254,
    'Newham': 1946157255,
    'Southwark': 1946157256,
    'Tower Hamlets': 1946157257,
    'Wandsworth': 1946157258,
    'Westminster': 1946157259,
    'Barking and Dagenham': 1946157260,
    'Barnet': 1946157261,
    'Bexley': 1946157262,
    'Brent': 1946157263,
    'Bromley': 1946157264,
    'Croydon': 1946157265,
    'Ealing': 1946157266,
    'Enfield': 1946157267,
    'Greenwich': 1946157268,
    'Harrow': 1946157269,
    'Havering': 1946157270,
    'Hillingdon': 1946157271,
    'Hounslow': 1946157272,
    'Kingston upon Thames': 1946157273,
    'Merton': 1946157274,
    'Redbridge': 1946157275,
    'Richmond upon Thames': 1946157276,
    'Sutton': 1946157277,
    'Waltham Forest': 1946157278,
    'Hammersmith & Fulham': 1946157249,
}

regions = {
    'Central London': ['City of London', 'Camden', 'Kensington and Chelsea', 'Islington', 'Westminster', 'Southwark', 'Lambeth'],
    'South London': ['Bromley', 'Croydon', 'Kingston upon Thames', 'Merton', 'Sutton', 'Wandsworth'],
    'East London': ['Barking and Dagenham', 'Bexley', 'Greenwich', 'Hackney', 'Havering', 'Lewisham', 'Newham', 'Tower Hamlets', 'Waltham Forest'],
    'North London': ['Barnet', 'Enfield', 'Haringey'],
    'West London': ['Brent', 'Ealing', 'Hammersmith & Fulham', 'Harrow', 'Hillingdon', 'Hounslow', 'Redbridge', 'Richmond upon Thames']
}

In [6]:
def compute_stats(freq_list_orig, freq_list_synth):
    """
    Compute different statistics (MAE, RMSE, SMRSE, R^2, and Pearson's correlation) on two frequency lists.

    Parameters
    ----------
    freq_list_orig: numpy.ndarray
        Frequency list for the original data
    freq_list_synth: numpy.ndarray
        Frequency list for the synthetic data

    Returns
    -------
    stat: dict
        Dictionary of the stats between the two lists
    """

    freq_list_orig, freq_list_synth = np.array(freq_list_orig), np.array(freq_list_synth)
    corr_mat = np.corrcoef(freq_list_orig, freq_list_synth)
    corr = corr_mat[0, 1]
    if np.isnan(corr): corr = 0.0
    # MAE
    mae = np.absolute(freq_list_orig - freq_list_synth).mean()
    # RMSE
    rmse = np.linalg.norm(freq_list_orig - freq_list_synth) / np.sqrt(len(freq_list_orig))
    # SRMSE
    freq_list_orig_avg = freq_list_orig.mean()
    srmse = rmse / freq_list_orig_avg
    # r-square
    u = np.sum((freq_list_synth - freq_list_orig) ** 2)
    v = np.sum((freq_list_orig - freq_list_orig_avg) ** 2)
    r2 = 1.0 - u / v
    stat = {'mae': mae, 'rmse': rmse, 'r2': r2, 'srmse': srmse, 'corr': corr}

    return stat

stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
tests = [ 'LPMC', 'DATGAN', 'ciDATGAN', 'ciDATGAN_ALL']
labels = ['NOMIS']
for t in tests:
    labels.append(t)
colrs = ['black', 'lightgrey', 'darkgrey', 'red', 'darkred']

colors = [matplotlib.colors.to_rgba(c) for c in colrs]

In [7]:
if not os.path.exists('./res_unobs/'):
    os.mkdir('./res_unobs/')

# Age

In [8]:
bins = [-np.inf, 16, 30, 45, 65, np.inf]
bin_labels = ['child', 'young', 'adult', 'old', 'senior']

if not os.path.exists('../../figures/regions/age/'):
    os.makedirs('../../figures/regions/age/')

In [11]:
res = {}

for t in tests:
    res[t] = {}
    for s in stats_str:
        res[t][s] = []

for r in tqdm(regions):

    nomis = {}

    for s in bin_labels:
        nomis[s] = 0

    for b in regions[r]:

        # Nomis data
        response = http.request("GET", "https://www.nomisweb.co.uk/api/v01/dataset/NM_503_1.data.csv?geography={}&rows=c_age&cols=rural_urban&measures=20100".format(boroughs[b]))

        with open('tmp.txt', 'w') as f:
            f.write(response.data.decode('utf-8'))

        df = pd.read_csv('tmp.txt')

        df = df.sort_values('C_AGE')
        df = df.iloc[1:, :]
        df.index = list(range(len(df)))

        df = df['GROUP 1: OBS_VALUE']

        dct = {'age': [df.iloc[5:17].sum(), df.iloc[17:31].sum(), df.iloc[31:46].sum(), df.iloc[46:66].sum(), df.iloc[66:].sum()]}

        tmp = pd.DataFrame(dct, index=bin_labels)

        for s in bin_labels:
            nomis[s] += tmp.loc[s][0]

    nomis = pd.DataFrame(nomis, index=['age']).T
    nomis /= nomis.sum()
    nomis = nomis['age']

    probs = [nomis]

    for t in tests:

        # Oversampled LPMC data
        df = pd.read_csv('../../data/synthetic/regions/{}/{}.csv'.format(t, r))
        df['age'] = pd.cut(df['age'], bins=bins, labels=bin_labels)
        df = df['age'].value_counts()
        df = df.sort_index()
        df /= df.sum()

        probs.append(df)

        sts = compute_stats(nomis, df)
        for s in stats_str:
            res[t][s].append(sts[s])

    dct = {}
    for l in labels:
        dct[l] = []

    idx = probs[0].index

    for i in idx:
        for j, l in enumerate(labels):
            try:
                val = probs[j].loc[i]
            except:
                val = 0
            dct[l].append(val)

    tmp_df = pd.DataFrame(dct, index=idx)

    tmp_df.plot(figsize=(10,7), kind='bar', color=colors)

    plt.legend()
    plt.xticks(rotation=45)
    plt.title('Age distribution for {}'.format(r))

    plt.savefig('../../figures/regions/age/{}.png'.format(r), bbox_inches='tight')
    plt.close()

  0%|          | 0/5 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpo

In [12]:
pickle.dump(res, open('./res_unobs/age.pkl', 'wb'))

## Number of people in household

In [13]:
if not os.path.exists('../../figures/regions/hh_people/'):
    os.makedirs('../../figures/regions/hh_people/')

In [17]:
res = {}

for t in tests:
    res[t] = {}
    for s in stats_str:
        res[t][s] = []

for r in tqdm(regions):

    nomis = np.zeros(8)

    for b in regions[r]:

        # Nomis data
        url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_538_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(boroughs[b])
        response = http.request('GET', url)

        with open('tmp.txt', 'w') as f:
            f.write(response.data.decode('utf-8'))

        df = pd.read_csv('tmp.txt')

        tmp = list(df['GROUP 1: OBS_VALUE'].iloc[1:])
        tmp = np.array(tmp)

        nomis += tmp

    nomis /= nomis.sum()

    probs = [nomis]

    for t in tests:

        df = pd.read_csv('../../data/synthetic/regions/{}/{}.csv'.format(t, r))
        tmp = df['hh_people'].value_counts().sort_index(ascending=True)
        arr = np.zeros(len(nomis))
        for i in tmp.index:
            if i-1 >= len(arr):
                arr[len(arr)-1] += tmp[i]
            else:
                arr[i-1] = tmp[i]
        arr /= arr.sum()

        probs.append(arr)

        sts = compute_stats(nomis, arr)
        for s in stats_str:
            res[t][s].append(sts[s])

    dct = {}
    for l in labels:
        dct[l] = []

    idx = range(1, len(nomis)+1)

    for i in idx:
        for j, l in enumerate(labels):
            try:
                val = probs[j][i-1]
            except:
                val = 0
            dct[l].append(val)

    tmp_df = pd.DataFrame(dct, index=idx)

    tmp_df.plot(figsize=(10,7), kind='bar', color=colors)

    plt.legend()
    plt.xticks(rotation=45)
    plt.title('Distribution of people per household for {}'.format(r))

    plt.savefig('../../figures/regions/hh_people/{}.png'.format(r), bbox_inches='tight')
    plt.close()

  0%|          | 0/5 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpo

In [18]:
pickle.dump(res, open('./res_unobs/hh_people.pkl', 'wb'))

## Distance travelled to work

In [19]:
bins = [-np.inf, 2000, 5000, 10000, 20000, 30000, 40000, 60000, np.inf]
bin_labels = ['<2km', '2-5km', '5-10km', '10-20km', '20-30km', '30-40km', '40-60km', '>60km']

if not os.path.exists('../../figures/regions/dist_work/'):
    os.makedirs('../../figures/regions/dist_work/')

In [20]:
res = {}

for t in tests:
    res[t] = {}
    for s in stats_str:
        res[t][s] = []

for r in tqdm(regions):

    nomis = {}

    for s in bin_labels:
        nomis[s] = 0

    for b in regions[r]:
        # Get the data
        url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_153_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(boroughs[b])
        response = http.request('GET', url)

        with open('tmp.txt', 'w') as f:
            f.write(response.data.decode('utf-8'))

        df = pd.read_csv('tmp.txt')
        df = df.sort_values('CELL')
        df.index = df['CELL']

        tmp = list(df['GROUP 1: OBS_VALUE'].iloc[1:9])

        for i, s in enumerate(bin_labels):
            nomis[s] += tmp[i]

    nomis = pd.DataFrame(nomis, index=['dist']).T
    nomis /= nomis.sum()
    nomis = nomis['dist']

    probs = [nomis]

    for t in tests:

        df = pd.read_csv('../../data/synthetic/regions/{}/{}.csv'.format(t, r))
        df['distance'] = pd.cut(df['distance'], bins=bins, labels=bin_labels)
        df = df[df['purpose'] == 'HBW']
        df = df[((df['age'] >= 16) & (df['age'] <= 74))]
        df = df['distance'].value_counts()
        df = df
        df = df.sort_index()
        df /= df.sum()

        probs.append(df)

        sts = compute_stats(nomis, df)
        for s in stats_str:
            res[t][s].append(sts[s])

    dct = {}
    for l in labels:
        dct[l] = []

    idx = probs[0].index

    for i in idx:
        for j, l in enumerate(labels):
            try:
                val = probs[j].loc[i]
            except:
                val = 0
            dct[l].append(val)

    tmp_df = pd.DataFrame(dct, index=idx)

    tmp_df.plot(figsize=(10,7), kind='bar', color=colors)

    plt.legend()
    plt.xticks(rotation=45)
    plt.title('Distribution for the distance to work for {}'.format(r))

    plt.savefig('../../figures/regions/dist_work/{}.png'.format(r), bbox_inches='tight')
    plt.close()

  0%|          | 0/5 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpo

In [21]:
pickle.dump(res, open('./res_unobs/dist_work.pkl', 'wb'))

## Method of travel to work

In [22]:
order = ['pt', 'drive', 'walk', 'cycle']

if not os.path.exists('../../figures/regions/mode_work/'):
    os.makedirs('../../figures/regions/mode_work/')

In [23]:
res = {}

for t in tests:
    res[t] = {}
    for s in stats_str:
        res[t][s] = []

for r in tqdm(regions):

    nomis = {}

    for s in order:
        nomis[s] = 0

    for b in regions[r]:

        # Get the data
        url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_568_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(boroughs[b])
        response = http.request('GET', url)

        with open('tmp.txt', 'w') as f:
            f.write(response.data.decode('utf-8'))

        df = pd.read_csv('tmp.txt')
        df = df.sort_values('CELL')
        df.index = df['CELL']

        values = df['GROUP 1: OBS_VALUE']
        nomis['drive'] += np.sum(values[6:9])
        nomis['pt'] += np.sum(values[2:6])
        nomis['walk'] += values[10]
        nomis['cycle'] += values[9]

    nomis = pd.DataFrame(nomis, index=['mode']).T
    nomis = nomis.sort_index()
    nomis /= nomis.sum()
    nomis = nomis['mode']

    probs = [nomis]

    for t in tests:
        df = pd.read_csv('../../data/synthetic/regions/{}/{}.csv'.format(t, r))
        df = df[df['purpose'] == 'HBW']
        df = df[((df['age'] >= 16) & (df['age'] <= 74))]
        df = df['travel_mode'].value_counts()
        for i in order:
            if i not in df.index:
                df[i] = 0

        df = df.sort_index()
        df /= df.sum()

        probs.append(df)

        sts = compute_stats(nomis, df)
        for s in stats_str:
            res[t][s].append(sts[s])

    dct = {}
    for l in labels:
        dct[l] = []

    idx = probs[0].index

    for i in idx:
        for j, l in enumerate(labels):
            try:
                val = probs[j].loc[i]
            except:
                val = 0
            dct[l].append(val)

    tmp_df = pd.DataFrame(dct, index=idx)

    tmp_df.plot(figsize=(10,7), kind='bar', color=colors)

    plt.legend()
    plt.xticks(rotation=45)
    plt.title('Distribution for model to work for {}'.format(r))

    plt.savefig('../../figures/regions/mode_work/{}.png'.format(r), bbox_inches='tight')
    plt.close()

  0%|          | 0/5 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpo

In [24]:
pickle.dump(res, open('./res_unobs/mode_work.pkl', 'wb'))

## Number of cars in the household

In [25]:
if not os.path.exists('../../figures/regions/hh_vehicles/'):
    os.makedirs('../../figures/regions/hh_vehicles/')

In [26]:
res = {}

for t in tests:
    res[t] = {}
    for s in stats_str:
        res[t][s] = []

for r in tqdm(regions):

    nomis = np.zeros(5)

    for b in regions[r]:

        # Get the data
        url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_621_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(boroughs[b])
        response = http.request('GET', url)

        with open('tmp.txt', 'w') as f:
            f.write(response.data.decode('utf-8'))

        df = pd.read_csv('tmp.txt')

        tmp = df['GROUP 1: OBS_VALUE'].iloc[1:6]
        tmp = np.array(tmp)
        nomis += tmp

    nomis /= nomis.sum()

    probs = [nomis]

    for t in tests:

        df = pd.read_csv('../../data/synthetic/regions/{}/{}.csv'.format(t, r))
        tmp = df['hh_vehicles'].value_counts().sort_index(ascending=True)
        arr = np.zeros(len(nomis))
        for i in tmp.index:
            if i-1 >= len(arr):
                arr[len(arr)-1] += tmp[i]
            else:
                arr[i-1] = tmp[i]
        arr /= arr.sum()

        probs.append(arr)

        sts = compute_stats(nomis, arr)
        for s in stats_str:
            res[t][s].append(sts[s])

    dct = {}
    for l in labels:
        dct[l] = []

    idx = range(1, len(nomis)+1)

    for i in idx:
        for j, l in enumerate(labels):
            try:
                val = probs[j][i-1]
            except:
                val = 0
            dct[l].append(val)

    tmp_df = pd.DataFrame(dct, index=idx)

    tmp_df.plot(figsize=(10,7), kind='bar', color=colors)

    plt.legend()
    plt.xticks(rotation=45)
    plt.title('Distribution of vehicles per household for {}'.format(r))

    plt.savefig('../../figures/regions/hh_vehicles/{}.png'.format(r), bbox_inches='tight')
    plt.close()

  0%|          | 0/5 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((
D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpo

In [27]:
pickle.dump(res, open('./res_unobs/hh_vehicles.pkl', 'wb'))